# Setup

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor

# Data Loading

In [3]:
train = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/30_Training Dataset_V2/training_data.csv') # training data
testpub = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/public_dataset.csv')                 # testing data (public leader board)
subpub = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/public_submission_template.csv')      # submission file

# Data Preprocessing

In [4]:
# Check missing value
print(train.isnull().sum())
print()
print(testpub.isnull().sum())

ID            0
縣市            0
鄉鎮市區          0
路名            0
土地面積          0
使用分區          0
移轉層次          0
總樓層數          0
主要用途          0
主要建材          0
建物型態          0
屋齡            0
建物面積          0
車位面積          0
車位個數          0
橫坐標           0
縱坐標           0
備註        11659
主建物面積         0
陽台面積          0
附屬建物面積        0
單價            0
dtype: int64

ID           0
縣市           0
鄉鎮市區         0
路名           0
土地面積         0
使用分區         0
移轉層次         0
總樓層數         0
主要用途         0
主要建材         0
建物型態         0
屋齡           0
建物面積         0
車位面積         0
車位個數         0
橫坐標          0
縱坐標          0
備註        5828
主建物面積        0
陽台面積         0
附屬建物面積       0
dtype: int64


In [5]:
train = train.drop(['備註'], axis=1)
testpub = testpub.drop(['備註'], axis=1)
print(train.isnull().sum())
print()
print(testpub.isnull().sum())

ID        0
縣市        0
鄉鎮市區      0
路名        0
土地面積      0
使用分區      0
移轉層次      0
總樓層數      0
主要用途      0
主要建材      0
建物型態      0
屋齡        0
建物面積      0
車位面積      0
車位個數      0
橫坐標       0
縱坐標       0
主建物面積     0
陽台面積      0
附屬建物面積    0
單價        0
dtype: int64

ID        0
縣市        0
鄉鎮市區      0
路名        0
土地面積      0
使用分區      0
移轉層次      0
總樓層數      0
主要用途      0
主要建材      0
建物型態      0
屋齡        0
建物面積      0
車位面積      0
車位個數      0
橫坐標       0
縱坐標       0
主建物面積     0
陽台面積      0
附屬建物面積    0
dtype: int64


In [6]:
# Check column Dtype
print(train.info())
print()
print(testpub.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11751 entries, 0 to 11750
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      11751 non-null  object 
 1   縣市      11751 non-null  object 
 2   鄉鎮市區    11751 non-null  object 
 3   路名      11751 non-null  object 
 4   土地面積    11751 non-null  float64
 5   使用分區    11751 non-null  object 
 6   移轉層次    11751 non-null  int64  
 7   總樓層數    11751 non-null  int64  
 8   主要用途    11751 non-null  object 
 9   主要建材    11751 non-null  object 
 10  建物型態    11751 non-null  object 
 11  屋齡      11751 non-null  float64
 12  建物面積    11751 non-null  float64
 13  車位面積    11751 non-null  float64
 14  車位個數    11751 non-null  float64
 15  橫坐標     11751 non-null  int64  
 16  縱坐標     11751 non-null  int64  
 17  主建物面積   11751 non-null  float64
 18  陽台面積    11751 non-null  float64
 19  附屬建物面積  11751 non-null  float64
 20  單價      11751 non-null  float64
dtypes: float64(9), int64(4), object(8)


In [7]:
numeric_features = ['土地面積', '移轉層次', '總樓層數', '屋齡', '建物面積', '車位面積', '車位個數', '主建物面積', '陽台面積', '附屬建物面積']
train_numeric = train[numeric_features]
testpub_numeric = testpub[numeric_features]

scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_numeric)
testpub_scaled = scaler.transform(testpub_numeric)

train[numeric_features] = train_scaled
testpub[numeric_features] = testpub_scaled

In [9]:
train_object_columns = train.select_dtypes(include=['object']).columns
testpub_object_columns = testpub.select_dtypes(include=['object']).columns

for column in train_object_columns:
    print(f"{column}, Dtype: {train[column].dtype}")
print()
for column in testpub_object_columns:
    print(f"{column}, Dtype: {testpub[column].dtype}")

ID, Dtype: object
縣市, Dtype: object
鄉鎮市區, Dtype: object
路名, Dtype: object
使用分區, Dtype: object
主要用途, Dtype: object
主要建材, Dtype: object
建物型態, Dtype: object

ID, Dtype: object
縣市, Dtype: object
鄉鎮市區, Dtype: object
路名, Dtype: object
使用分區, Dtype: object
主要用途, Dtype: object
主要建材, Dtype: object
建物型態, Dtype: object


In [10]:
train = pd.DataFrame(train)
tetspub = pd.DataFrame(testpub)

train['ID'] = train['ID'].apply(lambda x: int(x.replace('TR-', '')) if isinstance(x, str) and x.startswith('TR-') else x)
testpub['ID'] = testpub['ID'].apply(lambda x: int(x.replace('PU-', '')) if isinstance(x, str) and x.startswith('PU-') else x)

In [11]:
label_encoder = LabelEncoder()
object_columns = train.select_dtypes(include=['object']).columns

for column in object_columns:
    combined_data = pd.concat([train[column], testpub[column]], axis=0)
    label_encoder.fit(combined_data)

    # 將訓練集和測試集中的類別進行編碼
    train[column] = label_encoder.transform(train[column])
    testpub[column] = label_encoder.transform(testpub[column])

In [13]:
# Data Spliting
X = train.drop('單價', axis=1)
y = train['單價']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Model Training

In [14]:
# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.31285929290099446


In [15]:
# XGBoost
model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.10351293748630615


In [16]:
# H2O
! pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.42.0.3-py2.py3-none-any.whl size=240945068 sha256=92685fa238a4f82e10449caefd042a2b8662e05d6e6b8af2460ba08723251726
  Stored in directory: /root/.cache/pip/wheels/f9/7d/1d/e1258f0558603969a8393cb5b22a18e313337f01627a118302
Successfully built h2o


In [17]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphcm16zly
  JVM stdout: /tmp/tmphcm16zly/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmphcm16zly/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_unknownUser_abtxa2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [20]:
data = h2o.H2OFrame(train)
trainn, test, valid = data.split_frame([0.6, 0.2], seed=42)
y = '單價'
x = list(data.columns)
x.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [24]:
aml = H2OAutoML(max_models=30, max_runtime_secs=120, seed=1, sort_metric="mae")
aml.train(x=x, y=y, training_frame=trainn)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_3_20230923_95737


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    56                 56                          129062                 10           10           10            22            308           178.661

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.037956492081438295
RMSE: 0.19482425947873713
MAE: 0.11641547302790693
RMSLE: 0.05006484752394662
Mean Residual Deviance: 0.037956492081438295

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.10960467875732621
RMSE: 0.33106597342119926
MAE: 0.1986528003432589
RMSLE: 0.08551614743662617
Mean Residual Deviance: 0.10960467875732621

Cross-Validation Metrics Summary: 
                        mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     0.198776   0.00726051  0.189969      0.195447      0.208965      0.196875      0.202622
mean_residual_deviance  0.109809   0.0618361   0.0784821     0.0793329     0.0988907     0.0732945     0.219043
mse                     0.109809   0.0618361   0.0784821     0.0793329     0.0988907     0.0732945     0.219043
r2                      0.891024   0.0502263   0.909849      0.914049      0.908949      0.920712      0.801562
residual_deviance       0.109809   0.0618361   0.0784821     0.0793329     0.0988907     0.0732945     0.219043
rmse                    0.323005   0.0827364   0.280147      0.281661      0.314469      0.27073       0.468021
rmsle                   0.0854141  0.00570157  0.0806338     0.0817756     0.0849224     0.0846733     0.0950656

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-09-23 09:59:28  9.750 sec   0                  0.991006         0.730004        0.982094
    2023-09-23 09:59:28  9.971 sec   5                  0.647612         0.460241        0.419401
    2023-09-23 09:59:28  10.178 sec  10                 0.451758         0.306617        0.204085
    2023-09-23 09:59:29  10.402 sec  15                 0.346508         0.222764        0.120068
    2023-09-23 09:59:29  10.621 sec  20                 0.288557         0.178053        0.083265
    2023-09-23 09:59:29  10.862 sec  25                 0.256816         0.154911        0.0659546
    2023-09-23 09:59:29  11.216 sec  30                 0.237655         0.141075        0.0564801
    2023-09-23 09:59:30  11.541 sec  35                 0.222103         0.130323        0.0493299
    2023-09-23 09:59:30  11.659 sec  40                 0.212056         0.124339        0.0449677
    2023-09-23 09:59:30  11.769 sec  45                 0.203746         0.119881        0.0415126
    2023-09-23 09:59:30  11.884 sec  50                 0.198512         0.117821        0.0394072
    2023-09-23 09:59:30  11.977 sec  55                 0.195364         0.116464        0.0381669
    2023-09-23 09:59:30  11.996 sec  56                 0.194824         0.116415        0.0379565

Variable Importances: 
variable      relative_importance    scaled_importance    percentage
------------  ---------------------  -------------------  ------------
橫坐標        14913.5                1                    0.476942
縣市          6784.53                0.454924             0.216972
縱坐標        2336.97                0.156701             0.0747373
屋齡          2263.15                0.151751 

In [25]:
aml.leaderboard.head()

model_id,mae,rmse,mse,rmsle,mean_residual_deviance
GBM_4_AutoML_3_20230923_95737,0.198653,0.331066,0.109605,0.0855161,0.109605
GBM_3_AutoML_3_20230923_95737,0.203106,0.333298,0.111088,0.0865886,0.111088
GBM_2_AutoML_3_20230923_95737,0.205551,0.334197,0.111687,0.0878669,0.111687
DRF_1_AutoML_3_20230923_95737,0.20648,0.347016,0.12042,0.0888268,0.12042
GBM_1_AutoML_3_20230923_95737,0.216674,0.352276,0.124098,0.0922557,0.124098
XGBoost_1_AutoML_3_20230923_95737,0.223036,0.356484,0.127081,0.0958096,0.127081
XGBoost_2_AutoML_3_20230923_95737,0.224547,0.365666,0.133712,0.0963057,0.133712
GLM_1_AutoML_3_20230923_95737,0.547219,0.778749,0.60645,nan,0.60645


In [42]:
best_model = aml.leader
varimp = best_model.varimp()

varimp_df = pd.DataFrame(varimp, columns=["Feature", "Relative Importance", "Scaled Importance", "Percentage"])
print(varimp_df)

   Feature  Relative Importance  Scaled Importance  Percentage
0      橫坐標         14913.542969           1.000000    0.476942
1       縣市          6784.525879           0.454924    0.216972
2      縱坐標          2336.968018           0.156701    0.074737
3       屋齡          2263.150879           0.151751    0.072377
4     總樓層數           998.787170           0.066972    0.031942
5     鄉鎮市區           763.488892           0.051194    0.024417
6     車位面積           607.277222           0.040720    0.019421
7     土地面積           383.034271           0.025684    0.012250
8    主建物面積           356.457397           0.023902    0.011400
9     移轉層次           340.600616           0.022838    0.010893
10    建物面積           339.552063           0.022768    0.010859
11      路名           307.245422           0.020602    0.009826
12    主要建材           290.439331           0.019475    0.009288
13    陽台面積           141.570404           0.009493    0.004527
14      ID           136.098099           0.009126    0

# Test

In [26]:
subpub = pd.DataFrame(subpub)
pred = model.predict(testpub)
subpub['predicted_price'] = pred

In [31]:
datatp = h2o.H2OFrame(testpub)
pred = aml.leader.predict(datatp)
predd = pred.as_data_frame()
subpub['predicted_price'] = predd

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [32]:
subpub.to_csv('/content/drive/MyDrive/ME /SinoPac/submission_3.csv', index=False)